# 常规赛：视杯视盘分割（GAMMA挑战赛任务三）
## 任务描述
给定一组分割数据，训练分割模型。
## 技术路线
通过PaddleSeg调用Unet训练三分类模型。
## 说明
本项目包含了截止到2021年11月3日为止的测试记录，对应的模型信息参考测试记录。

# 测试记录

如果你喜欢我的测试记录，并认为这个项目和记录对你有帮助，欢迎在之后的版本中也更新并公开你的测试记录~

|Index| 版本 | score | disc | cup | vCDR | 备注 |
| --------| -------- | -------- | -------- | -------- | -------- | -------- | 
|0| 版本0     |7.00177	|0.86745	|0.74444	|0.09006	    | 仅使用Resize, Norm, 训练100次     |
|1| 版本0     |7.91845|	0.91196	|0.83259	|0.05122	    | 增加了transforms, 训练4000次时崩溃，备注此次训练结果为1     |
|2| 版本1     |7.98176|	0.9226	|0.83887	|0.05063		    | 在1基础上训练，保存为2     |
|3| 版本2     |	8.13688	|0.93803|	0.85106	|0.04673		    | 在2基础上训练，重新划分数据集，保存为3     |
|4| 版本2     |	8.11225	|0.93858|	0.84898	|0.04789		    | 在3基础上训练。<font color=#ff0000>效果比3差，可能这就已经是这种模式下的极限了</font>     |
|5| 版本3     |	7.26362	|0.87501|	0.76456	|0.0747			    | 更替了Resize的大小为1000*1500，效果不是特别好，保存为4     |

# 正式开始
## 获取数据

In [ ]:
! wget https://dataset-bj.cdn.bcebos.com/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/task3_disc_cup_segmentation.zip

In [ ]:
! unzip -oq /home/aistudio/task3_disc_cup_segmentation.zip

## 准备包

In [1]:
!pip install paddleseg

#导入常用的库
import os
import random
import numpy as np
from random import shuffle
import cv2
import paddle
from PIL import Image
import shutil
import re
from paddle.vision.transforms import functional as F
import os.path
import paddleseg.transforms as T
from paddleseg.datasets import OpticDiscSeg,Dataset
import paddleseg.models
from paddleseg.models import UNet
from paddleseg.models import OCRNet
from paddleseg.models.losses import CrossEntropyLoss,DiceLoss, MixedLoss
from paddleseg.core import train
from paddleseg.core import evaluate
from paddleseg.core import predict
from PIL import Image

## 数据转化

PaddleSeg需要使用txt文档指定训练信息，同样要求训练标签从0,1,2....开始编号。

In [ ]:
# 转化训练标签到指定格式
! mkdir training/mask
for item in os.listdir('training/Disc_Cup_Mask'):
    im=Image.open('training/Disc_Cup_Mask/'+item)
    im=(np.array(im).astype(float)/255*2).astype(int).astype('uint8')
    im = Image.fromarray(im)
    im.save('training/mask/'+item)

mkdir: cannot create directory ‘training/mask’: File exists


In [ ]:
# 观察图片尺寸
names = [item.split('.')[0] for item in os.listdir('training/fundus_color_images')]
img_dir='/home/aistudio/training/fundus_color_images/'
mask_dir='/home/aistudio/training/mask/'
for name in names:
    im=Image.open(img_dir+name+'.jpg')
    print(np.array(im).shape)

In [ ]:
# 生成对应的图片-标签数据
try: 
    os.rename('/home/aistudio/training/fundus color images/','/home/aistudio/training/fundus_color_images/')
except:
    print('have renamed')
names = [item.split('.')[0] for item in os.listdir('training/fundus_color_images')]
ori_train_list=[]
img_dir='/home/aistudio/training/fundus_color_images/'
mask_dir='/home/aistudio/training/mask/'
for name in names:
    if os.path.exists(img_dir+name+'.jpg') and os.path.exists(mask_dir+name+'.png'):
        ori_train_list.append(img_dir+name+'.jpg '+mask_dir+name+'.png'+'\n')

have renamed


In [ ]:
# 生成txt文档
cut_percent=0.1
cut_point=int(cut_percent*len(ori_train_list))
random.shuffle(ori_train_list)

with open('train.txt','w') as f:
    for item in ori_train_list[cut_point:]:
        f.write(item)

with open('eval.txt','w') as f:
    for item in ori_train_list[:cut_point]:
        f.write(item)

## 训练

这里直接使用了1000*1500的Resize大小，可以根据测试记录找到自己想要的Reszie格式

In [2]:
# 定义transforms
train_transforms = [
    T.RandomHorizontalFlip(),# 水平翻转
    T.RandomVerticalFlip(),# 垂直翻转
    T.RandomDistort(0.4),# 随机扭曲
    T.RandomBlur(0.3),# 高斯模糊
    T.RandomScaleAspect(min_scale=0.5,aspect_ratio=0.5),# 随机缩放
    T.Resize(target_size=(1000,1500)),
    T.Normalize()  # 图像标准化
]

val_transforms = [
    T.Resize(target_size=(1000,1500)),
    T.Normalize()
]

In [4]:
# 构建训练集
train_dataset = Dataset(
    dataset_root='/home/aistudio',
    train_path='/home/aistudio/train.txt',
    transforms=train_transforms,
    num_classes=3,
    mode='train'
)
#验证集
val_dataset = Dataset(
    dataset_root='/home/aistudio',
    val_path='/home/aistudio/eval.txt',
    transforms=val_transforms,
    num_classes=3,
    mode='val'
)

In [5]:
model = UNet(num_classes=3)
# model = UNet(num_classes=3,pretrained='output/UNet/3/model.pdparams')#使用预训练模型unet进行训练

W1102 09:29:32.724684   143 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1102 09:29:32.730556   143 device_context.cc:422] device: 0, cuDNN Version: 7.6.


In [6]:
base_lr =0.001
iters = 10000

#自动调整学习率
lr =paddle.optimizer.lr.CosineAnnealingDecay(base_lr, T_max=(iters // 3), last_epoch=0.5)
u_optimizer = paddle.optimizer.Adam(lr, parameters=model.parameters())

mixtureLosses = [CrossEntropyLoss(),DiceLoss() ]
mixtureCoef = [0,1]
losses = {}
losses['types'] = [MixedLoss(mixtureLosses, mixtureCoef)]
losses['coef'] = [1]

In [7]:
#进行训练
train(
    model = model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=u_optimizer,
    save_dir='output/UNet',
    iters=iters,  
    batch_size=3, 
    save_interval=2000,
    log_iters=20,
    num_workers=2,
    losses=losses,
    # losses=CrossEntropyLoss(),
    use_vdl=False
    )

## 预测

In [8]:
test_list=["testing/fundus color images/"+item for item in os.listdir("testing/fundus color images")]

transforms = T.Compose([
    T.Resize(target_size=(1000, 1500)),
    T.Normalize()
])

model = paddleseg.models.UNet(num_classes=3)

predict(
    model,
    #换自己保存的模型文件
    model_path = 'output/UNet/best_model/model.pdparams',
    transforms=transforms,
    image_list=test_list,
    save_dir='Predict_result',)

2021-11-02 14:22:55 [INFO]	Loading pretrained model from output/UNet/best_model/model.pdparams
2021-11-02 14:22:55 [INFO]	There are 112/112 variables loaded into UNet.
2021-11-02 14:22:55 [INFO]	Start to predict...


100/100 [==============================] - 46s 457ms/st


## 预测结果打包

需要将预测结果转化为指定格式并打包提交

In [9]:
! mkdir Disc_Cup_Segmentations
for img in os.listdir('Predict_result/pseudo_color_prediction'):
    img_dir='Predict_result/pseudo_color_prediction/'+img
    im=Image.open(img_dir)
    im=Image.fromarray((np.array(im).astype(float)*255/2).round().astype('uint8'))
    img=img.split('.')[0]
    im.save('Disc_Cup_Segmentations/'+img+'.bmp')

mkdir: cannot create directory ‘Disc_Cup_Segmentations’: File exists


In [10]:
! zip -q -r result.zip Disc_Cup_Segmentations

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 